In [1]:
import pandas as pd
import transbigdata as tbd 
import geopandas as gpd 
import warnings 
warnings.filterwarnings('ignore')

In [3]:
# 读取修正版的顺易充充电站数据
all_shunyi_station = gpd.read_file('./data/顺易充数据/修正_all_stations_wgs84.json')
# 读取上海的行政区划
shanghai_admin = tbd.getadmin('上海市', ak = '169521e66b7a30adcaa36ae0a9c04c8d', jscode='2cdd826ad4099fae293dbabf3657c78d', subdistricts=False, timeout=20)
shanghai_shunyi_station = gpd.sjoin(all_shunyi_station,shanghai_admin)
# 读取中促盟的充电站信息
zhongcumeng_station = gpd.read_file('./data/上海充电桩_中促盟_合并站点.geojson')
shanghai_shunyi_station.rename(columns={'index_right':'index'},inplace=True)

In [8]:
corresponding_station = tbd.ckdnearest_point(shanghai_shunyi_station,zhongcumeng_station)

#### 找到中促盟和顺易充对应的充电站

In [28]:
corresponding_station = tbd.ckdnearest_point(shanghai_shunyi_station,zhongcumeng_station)
corresponding_station = corresponding_station[['stationId','myid']]
station_agg = pd.merge(shanghai_shunyi_station,corresponding_station,on=['stationId'],how='inner')

In [31]:
station_agg = station_agg[['stationId','myid']]
# 保存对应关系
station_agg.to_csv('./output/shunyip_to_hw.csv',index=None)

#### 整合数据到一起

In [35]:
# 读取中促盟特征数据
feature_zcm = pd.read_csv('./output/中促盟充电站特征.csv')
# 保留有用的列
feature_zcm = feature_zcm[['myid','充电站所处道路属性','充电站所属区域分类', '充电站类型','交流桩数量', '直流桩数量', '交流桩总装机功率', '直流桩总装机功率'
                           ,'area', 'technology_park', 'car_park', 'supermarket', 'residential','commercial','station_sum']]
# 读取对应关系，保留有用的中促盟对应站点信息
mapping_table= pd.read_csv('./output/shunyip_to_hw.csv')
feature_zcm_agg = pd.merge(mapping_table,feature_zcm,on=['myid'],how='inner')

In [40]:
feature_zcm_agg.to_csv('./output/站点及训练数据.csv',index=None)